In [ ]:
# Imports iniciais
import numpy as np
import matplotlib.pyplot as plt
import joblib
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
import ipywidgets as widgets
from IPython.display import display, clear_output
import itertools

# === CARREGAR MODELO TREINADO ===
modelo = joblib.load('modelo_4_camadas_ESRS_ESRD_COMPLETO.pkl')
weights = modelo['weights']
scaler = modelo['scaler']

# === FUNÇÕES RNA ===
def tanh(x): return np.tanh(x)
def neural_network(weights, X):
    W1 = weights[:90].reshape((9, 10))
    b1 = weights[90:100]
    W2 = weights[100:180].reshape((10, 8))
    b2 = weights[180:188]
    W3 = weights[-9:-1].reshape((8, 1))
    b3 = weights[-1]
    hidden1 = tanh(np.dot(X, W1) + b1)
    hidden2 = tanh(np.dot(hidden1, W2) + b2)
    output = np.dot(hidden2, W3) + b3
    return output.flatten()

def prever_deformacao_customizada(valores):
    entrada = np.array([valores])
    entrada_norm = scaler.transform(entrada)
    return neural_network(weights, entrada_norm)[0]

def corrigir_mr_para_Tc(E25_mpa, Tc):
    return E25_mpa * np.exp(-0.08 * (Tc - 25))

def calcular_confiabilidade_rosenblueth(media_inputs, covs, confianca):
    indices = {
        'revest_MR': 2, 'revest_h': 3,
        'base_MR': 4, 'base_h': 5,
        'subbase_MR': 6, 'subbase_h': 7,
        'subleito_MR': 8
    }
    mu_vector = media_inputs.copy()
    cov_values = [covs[k] / 100 for k in indices]
    combinacoes = list(itertools.product([-1, 1], repeat=7))
    resultados = []
    for sinais in combinacoes:
        v = mu_vector.copy()
        for i, key in enumerate(indices):
            v[indices[key]] *= (1 + sinais[i] * cov_values[i])
        resultados.append(prever_deformacao_customizada(v))
    media = np.mean(resultados)
    desvio = np.std(resultados)
    z = stats.norm.ppf(confianca / 100)
    return media, media + z * desvio

def calcular_C(ev):
    if ev >= 1.765e-3:
        return (0.00414131 / ev) ** 8.1
    else:
        log_C = (-0.1638 + 185.19 * ev) ** (-0.60586)
        return 10 ** log_C

def calcular_N(a, b, L):
    return (1 + (b * L / 200)) * a * L

def calcular_cp(c, w, pos, std):
    return sum(stats.norm.cdf(c + w/2, p, std) - stats.norm.cdf(c - w/2, p, std) for p in pos)

def obter_pc_por_faixa(xk_centro, w, t, Ne, h_total, std):
    f = np.linspace(-40 * 25.4, 40 * 25.4, 81)
    if Ne == 1 or h_total >= t - w:
        weq = w + t + h_total if Ne > 1 else w + h_total
        pos = [xk_centro]
    else:
        weq = w + h_total
        d = t / 2
        pos = [xk_centro - d, xk_centro + d]
    pos += [-p for p in pos]
    cp_vals = np.array([calcular_cp(fi, weq, pos, std) for fi in f])
    return f, 1 / cp_vals

# === AERONAVES ===
aeronaves = [
    {'nome': 'A-320', 'w': 31.5, 't': 92.69, 'xk_centro': 333.15, 'Ne': 2, 'pressao': 1.6, 'rodas': 1.4},
    {'nome': 'A-321', 'w': 34.7, 't': 92.71, 'xk_centro': 379.5, 'Ne': 2, 'pressao': 1.6, 'rodas': 1.4},
    {'nome': 'B-737', 'w': 32.3, 't': 86.36, 'xk_centro': 285.75, 'Ne': 2, 'pressao': 1.7, 'rodas': 1.5},
    {'nome': 'EMB-195', 'w': 29.2, 't': 86.36, 'xk_centro': 297.18, 'Ne': 2, 'pressao': 1.5, 'rodas': 1.4},
    {'nome': 'ATR-72', 'w': 22.6, 't': 43.60, 'xk_centro': 205.0, 'Ne': 2, 'pressao': 0.8, 'rodas': 1.2},
    {'nome': 'CESSNA', 'w': 16.9, 't': 0.0, 'xk_centro': 208.28, 'Ne': 1, 'pressao': 0.5, 'rodas': 1.0},
]

# === INTERFACE BÁSICA
print("Notebook carregado com sucesso. As funções e dados estão prontos para integração com widgets Voilà.")
